In [1]:
import pandas as pd
import numpy as np
from scipy.stats import kendalltau
from sklearn.preprocessing import LabelEncoder
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings('ignore')

### README

This notebook takes in user's daily bank balance data and generates user-level features.
User features are then joined with subset labels, and two dataframes are returned:
1) Dataframe for training of growth subset classification model (growth vs non-growth labels): 'user_features_final_growth'
2) Dataframe for training of stable subset classification model (stable vs non-stable labels): 'user_features_final_stable'

### Load files

In [2]:
# Users' daily balance time series data for the first 90 days
# Columms: pt_date, user_id, total_balance
train = pd.read_csv("train_data_l90d_daily_balance.csv").sort_values(by=['pt_date', 'user_id'])

# User labels generated from subsetting methodology on actual 180 days data
# For 'label' column: 0=neither; 1=stable; 2=growth
user_labels = pd.read_csv("user_subset_label.csv")

### Feature engineering pt 1: generating user features

For new features, add a function below (e.g., get_user_features_x) and update get_all_user_features function

In [3]:
# Normalized beta
def get_user_features(train):
    train_1 = train.copy().sort_values(by=["user_id", "pt_date"]).reset_index(drop=True)
    
    # Daily total balance
    agg_balance = train_1.groupby("pt_date").agg(
        total_balance=pd.NamedAgg(column="total_balance", aggfunc="sum")
    )
    agg_balance.reset_index(inplace=True)

    def z_score_normalize(column):
        mean = column.mean()
        std = column.std()
        normalized_column = (column - mean) / std
        return normalized_column

    # Z-score normalise daily total balance and obtain variance
    agg_balance["total_balance_normalized"] = z_score_normalize(
        agg_balance["total_balance"]
    )
    agg_balance_normalized_var = agg_balance["total_balance_normalized"].std() ** 2

    # Loop through users to calculate normalized beta
    user_id_list = []
    beta_normalized_list = []

    row = 0
    for user in train["user_id"].unique():
        user_df = train.iloc[row : row + 91]
        row += 91

        # calculate beta but with normalized values
        user_df.reset_index(inplace=True, drop=True)
        user_df = user_df[["total_balance"]]
        user_df["total_balance_normalized"] = z_score_normalize(
            user_df["total_balance"]
        )
        covariance_normalized = user_df["total_balance_normalized"].cov(
            agg_balance["total_balance_normalized"]
        )
        beta_normalized_list.append(covariance_normalized / agg_balance_normalized_var)

        user_id_list.append(user)

    data = {"user_id": user_id_list, "beta_normalized": beta_normalized_list}

    user_features = pd.DataFrame(data)
    return user_features
    
    

# User's first, last and average balance + last n-day EMA
# Standard deviation of balance, CV and stability index
def get_user_features_2(train, user_features):
    user_features_2 = user_features.copy()

    # Calculate first, last and average values of total_balance
    user_features_2['avg_balance'] = train.groupby('user_id')['total_balance'].mean().reset_index(drop=True)
    user_features_2['first_day_balance'] = train.groupby('user_id')['total_balance'].first().values
    user_features_2['last_day_balance'] = train.groupby('user_id')['total_balance'].last().values

    # Binning of user balance
    bins = [-1, 5000, 30000, 100000, 350000, 2000000, float('inf')]
    labels = ['Small', 'Medium', 'Big', 'Wealth Banking', 'Privilege Banking', 'Privilege Reserve']
    user_features_2['label_by_avg_bal'] = pd.cut(user_features_2['avg_balance'], bins=bins, labels=labels)
    user_features_2['label_by_first_bal'] = pd.cut(user_features_2['first_day_balance'], bins=bins, labels=labels)
    user_features_2['label_by_last_bal'] = pd.cut(user_features_2['last_day_balance'], bins=bins, labels=labels)
    
    # Standard Deviation, Coefficient of Variation and Stability Index
    user_features_2['volatility_stdev'] = train.groupby('user_id')['total_balance'].std().reset_index(drop=True)
    user_features_2['volatility_cv'] = (user_features_2['volatility_stdev'] / user_features_2['avg_balance']) * 100
    
    cv_scaled = (user_features_2['volatility_cv'] - min(user_features_2['volatility_cv'])) / \
    (max(user_features_2['volatility_cv']) - min(user_features_2['volatility_cv']))
    user_features_2['stability_index'] = 1 - cv_scaled
    
    
    # Calculate 7-day EMA, 20-day EMA, 50-day EMA and 90-day EMA (feature is the LAST n-day EMA)
    ema_spans = [7, 20, 50, 90]
    for span in ema_spans:
        user_features_2[f'ema_{span}day'] = train.groupby('user_id')['total_balance'].apply(lambda x: x.ewm(span=span).mean().iloc[-1]).values

    user_features_2.reset_index(drop=True, inplace=True)
    
    return user_features_2



# Binary feature of whether user has statistically significant +ve kendall coefficient, 
# Indicates whether there is a positive correlation between date & balance
# statistically significant (1% sig. level) & positive correlation -> positive trend
def get_user_features_3(train, user_features_2):
    kendall_coeff = train.groupby('user_id').apply(
        lambda group: kendalltau(group['pt_date'], group['total_balance'])[0]).reset_index()

    kendall_coeff.columns = ['user_id', 'kendall_coeff']

    kendall_p = train.groupby('user_id').apply(
        lambda group: kendalltau(group['pt_date'], group['total_balance'])[1]).reset_index()

    kendall_p.columns = ['user_id', 'kendall_p']

    kendall = kendall_coeff.merge(kendall_p, on='user_id')
    kendall['stat_sig_positive_kendall'] = np.where((kendall['kendall_p'] < 0.01) & \
                                                    (kendall['kendall_coeff'] > 0), 1, 0)

    user_features_3 = user_features_2.merge(kendall[['user_id', 'stat_sig_positive_kendall']], on='user_id')

    return user_features_3



# Number of times user's short-term EMA crosses long-term EMA
def get_user_features_4(train, user_features_3):
    train_2 = train.copy().sort_values(by=['user_id', 'pt_date'])

    # Calculate 7-day and 50-day EMA for each user
    train_2['ema_7day'] = train_2.groupby('user_id')['total_balance'].transform(lambda x: x.ewm(span=7).mean())
    train_2['ema_50day'] = train_2.groupby('user_id')['total_balance'].transform(lambda x: x.ewm(span=50).mean())

    # Detect EMA crossings by checking for changes in the sign of the difference
    train_2['ema_difference'] = train_2['ema_7day'] - train_2['ema_50day']
    train_2['ema_crosses'] = train_2['ema_difference'] * train_2['ema_difference'].shift(1) < 0 

    # Count the number of crossings for each user
    train_2['num_ema_crosses'] = train_2.groupby('user_id')['ema_crosses'].transform('sum')

    # User-level dataframe
    num_ema_crosses = train_2.groupby('user_id')['num_ema_crosses'].mean().reset_index()
    
    user_features_4 = user_features_3.merge(num_ema_crosses, on='user_id')
    return user_features_4


# Binary feature: whether user's bank balance is stationary
# ADF test on user balance at 1% significance level
def get_user_features_5(train, user_features_4):
    train_3 = train.copy().sort_values(by=['user_id', 'pt_date'])
    train_3['balance_change'] = train_3.groupby('user_id')['total_balance'].diff()

    # ADF test -> return both coefficient and p-value
    # H0: non-stationary

    def adf_test(data):
        result = adfuller(data['balance_change'].dropna())
        adf_coefficient = result[0]
        p_value = result[1]
        return pd.Series({'adf_coeff': adf_coefficient, 'adf_p': p_value})

    adf = train_3.groupby('user_id').apply(adf_test).reset_index()

    # Binary variable for stationarity
    adf['stationary'] = np.where(adf['adf_p'] < 0.01, 1, 0)
    adf.columns = ['user_id', 'adf_coeff', 'adf_p', 'stationary']
    
    user_features_5 = user_features_4.merge(adf[['user_id', 'stationary']], on='user_id')
    return user_features_5


# Features relating to recurring transactions
# Attempts to identify user traits i.e., what they are using their Maribank account for
# Didn't explore time interval between recurring transactions;
# Solely inferring purpose of usage based on recurring deposit/withdrawal amount for now
# Thresholds are also subjective
def get_user_features_6(train, user_features_5):

    train_4 = train.copy().sort_values(by=['user_id', 'pt_date'])
    train_4['balance_change'] = train_4.groupby('user_id')['total_balance'].diff()

    # Round off the balance change to 2dp; 
    train_4['balance_change'] = round(train_4['balance_change'], 2)
    
    grouped = train_4[['user_id', 'balance_change']].groupby('user_id')['balance_change']
    
    user_data = []

    for user, transactions in grouped:

        # Count transaction amounts (excluding 0 balance change) and their frequencies
        unique_transactions = transactions[transactions != 0].value_counts()

        # Filter for recurring transactions, where count > 1
        recurring_transactions = unique_transactions[unique_transactions > 1]
        recurring_tx_count = (unique_transactions > 1).sum()
        recurring_tx_amounts = []

        if recurring_tx_count > 0:
            num_distinct_recurring_tx = len(recurring_transactions)
            
            # List of recurring transactin for each user
            recurring_tx_amounts = recurring_transactions.index.tolist()
            
        else:
            num_distinct_recurring_tx = 0


        user_data.append({'user_id': user, 
                          'num_distinct_recurring_tx': num_distinct_recurring_tx, 
                          'recurring_tx_amounts': recurring_tx_amounts,
                         })

    recurring_tx = pd.DataFrame(user_data)
    
    # Binary feature: whether user has recurring transactions
    recurring_tx['recurring_transactions'] = np.where(recurring_tx['num_distinct_recurring_tx'] > 0, 1, 0)
    
    # Binary feature: whether user has recurring deposits
    has_recurring_deposits = [any(value > 0 for value in row) for row in recurring_tx['recurring_tx_amounts']]
    recurring_tx['recurring_deposits'] = np.where(has_recurring_deposits, 1, 0)
    
    # Binary feature: whether user has recurring withdrawals
    has_recurring_withdrawals = [any(value < 0 for value in row) for row in recurring_tx['recurring_tx_amounts']]
    recurring_tx['recurring_withdrawals'] = np.where(has_recurring_withdrawals, 1, 0)
    
    # Binary feature: whether user utilises bank acc for income
    # Income defined as recurring deposits of > $1000
    has_income = [any(value > 1000 for value in row) for row in recurring_tx['recurring_tx_amounts']]
    recurring_tx['income'] = np.where(has_income, 1, 0)
    
    # Binary feature: whether user utilises bank acc to pay for subscriptions
    # Subscription defined as recurring withdrawals of between 0 to $100
    has_subscription = [any(-100 <= value < 0 for value in row) for row in recurring_tx['recurring_tx_amounts']]
    recurring_tx['subscription'] = np.where(has_subscription, 1, 0)
    
    recurring_tx = recurring_tx.drop('recurring_tx_amounts', axis=1)
    
    user_features_6 = user_features_5.merge(recurring_tx, on='user_id')
    return user_features_6


# Absolute balance change mean and SD
# Transaction, withdrawal, deposit counts and proportions
def get_user_features_7(train, user_features_6):
    train_5 = train.copy().sort_values(by=["user_id", "pt_date"])
    train_5["abs_bal_change"] = train_5.groupby("user_id")["total_balance"].diff()
    
    train_5["pct_bal_change"] = train_5.groupby("user_id")["total_balance"].pct_change() * 100
    train_5["deposit_flag"] = np.where(train_5["pct_bal_change"] > 0.015, 1, 0)
    train_5["withdrawal_flag"] = np.where(train_5["pct_bal_change"] < -0.015, 1, 0)
    train_5["transaction_flag"] = np.where((train_5["deposit_flag"] == 1) | (train_5["withdrawal_flag"] == 1),
                                           1, 0)
    
    result = train_5.groupby("user_id").agg(
        abs_bal_change_mean=("abs_bal_change", "mean"),
        abs_bal_change_std=("abs_bal_change", "std"),
        transactions=("transaction_flag", "sum"),
        deposits=("deposit_flag", "sum"),
        withdrawals=("withdrawal_flag", "sum")
    ).reset_index()
    
    # Set proportion = 0 if there are no transactions, otherwise calculate proportion
    result['withdrawal_propn'] = np.where(result['transactions'] == 0, 0, result['withdrawals'] / result['transactions'])
    result['deposit_propn'] = np.where(result['transactions'] == 0, 0, result['deposits'] / result['transactions'])

    user_features_7 = user_features_6.merge(result, on='user_id')
    return user_features_7



# Growth coefficient (line of best fit of user balance) and trend
def get_user_features_8(train, user_features_7):
    train_6 = train.copy().sort_values(by=['user_id', 'pt_date'])

    def get_growth_coeff(data):
        x = list(range(0, 91))
        y = data['total_balance']
        a, b = np.polyfit(x, y, 1)
        return pd.Series({'growth_coeff': a})

    result = train_6.groupby('user_id').apply(get_growth_coeff).reset_index()
    
    trend_bins = [-float('inf'), -10, 10, float('inf')]
    trend_labels = ['Decreasing', 'Stable', 'Increasing']
    result['trend'] = pd.cut(result['growth_coeff'], bins=trend_bins, labels=trend_labels)
    
    user_features_8 = user_features_7.merge(result, on='user_id')
    return user_features_8
    

### Feature engineering pt2: encoding for categorical features

In [4]:
def identify_categorical_columns(user_features):
    categorical_cols = []
    for col in user_features.columns:
        if (user_features[col].dtype == 'object' or user_features[col].dtype == 'category') and col != 'user_id':
            categorical_cols.append(col)
    
    print("The categorical variables to be encoded are")
    print(categorical_cols)
    
    return categorical_cols


def encode_categorical_features(user_features):
    categorical_cols = identify_categorical_columns(user_features)
    
    label_encoder = LabelEncoder()
    for col in categorical_cols:
        user_features[col] = label_encoder.fit_transform(user_features[col])
        
    return user_features

### Dataframe with all user features (w/o subset labels)

In [5]:
def get_all_user_features(train):
    print("Generating user_features...")
    user_features = get_user_features(train)
    
    print("Generating user_features_2...")
    user_features_2 = get_user_features_2(train, user_features)
    
    print("Generating user_features_3...")
    user_features_3 = get_user_features_3(train, user_features_2)
    
    print("Generating user_features_4...")
    user_features_4 = get_user_features_4(train, user_features_3)

    print("Generating user_features_5...")
    user_features_5 = get_user_features_5(train, user_features_4)
    
    print("Generating user_features_6...")
    user_features_6 = get_user_features_6(train, user_features_5)
    
    print("Generating user_features_7...")
    user_features_7 = get_user_features_7(train, user_features_6)
    
    print("Generating user_features_8...")
    user_features_8 = get_user_features_8(train, user_features_7)
    
    print("Encoding categorical variables...")
    user_features_final = encode_categorical_features(user_features_8)
    
    return user_features_final

    return user_features

### Join with subset labels, obtain 2 dataframes for growth & stable classification models respectively

In [6]:
def label_users(user_labels, user_features_final):
    stable_labels = user_labels[['user_id', 'is_in_stable_subset']]\
    .rename(columns={'is_in_stable_subset': 'label'})
    
    growth_labels = user_labels[['user_id', 'is_in_growth_subset']]\
    .rename(columns={'is_in_growth_subset': 'label'})
    
    print("Labelling users...")
    user_features_final_stable = stable_labels.merge(user_features_final, on='user_id')
    user_features_final_growth = growth_labels.merge(user_features_final, on='user_id')
    
    return user_features_final_stable, user_features_final_growth

### Final function

Takes about 8 minutes to run

In [7]:
# Arguments:
# train: last 90 days bank balance data
# user_features: initial features exported from oliver's EDA code
# user_labels: labels generated from actual next 180 days data (subsetting methodology)

def get_user_subsets_w_features(train, user_labels):
    user_features_final = get_all_user_features(train)
    
    # Dataframe with entire dataset's users
#     df = user_features_final.merge(user_labels[['user_id', 'text_label', 'label']], on='user_id')
#     df.to_csv("user_features_l90_labelled_20231111.csv", index=False)
    
    # Split into 2 dataframes
    # user_features_final_stable: stable & non-stable users; user_features_final_growth: growth & non-growth users
    user_features_final_stable, user_features_final_growth = label_users(user_labels, user_features_final)
    
    print("Done!")
    return user_features_final_stable, user_features_final_growth


user_features_final_stable, user_features_final_growth = get_user_subsets_w_features(train, user_labels)

Generating user_features...
Generating user_features_2...
Generating user_features_3...
Generating user_features_4...
Generating user_features_5...
Generating user_features_6...
Generating user_features_7...
Generating user_features_8...
Encoding categorical variables...
The categorical variables to be encoded are
['label_by_avg_bal', 'label_by_first_bal', 'label_by_last_bal', 'trend']
Labelling users...
Done!


In [8]:
user_features_final_stable.columns

Index(['user_id', 'label', 'beta_normalized', 'avg_balance',
       'first_day_balance', 'last_day_balance', 'label_by_avg_bal',
       'label_by_first_bal', 'label_by_last_bal', 'volatility_stdev',
       'volatility_cv', 'stability_index', 'ema_7day', 'ema_20day',
       'ema_50day', 'ema_90day', 'stat_sig_positive_kendall',
       'num_ema_crosses', 'stationary', 'num_distinct_recurring_tx',
       'recurring_transactions', 'recurring_deposits', 'recurring_withdrawals',
       'income', 'subscription', 'abs_bal_change_mean', 'abs_bal_change_std',
       'transactions', 'deposits', 'withdrawals', 'withdrawal_propn',
       'deposit_propn', 'growth_coeff', 'trend'],
      dtype='object')

In [9]:
user_features_final_stable

,user_id,label,beta_normalized,avg_balance,first_day_balance,last_day_balance,label_by_avg_bal,label_by_first_bal,label_by_last_bal,volatility_stdev,...,subscription,abs_bal_change_mean,abs_bal_change_std,transactions,deposits,withdrawals,withdrawal_propn,deposit_propn,growth_coeff,trend
0,0000102c03057f91c90faa1011e59870f2f8710597c27b...,0,0.104897,74243.627143,167126.280,76336.380,0,5,0,60422.271064,...,0,-1008.776667,28910.758769,63,7,56,0.888889,0.111111,62.104009,1
1,0001c00642af4db4bfab49d3884b5e88f72acddc4cb03b...,0,-0.010642,11684.816374,13194.525,10103.580,1,1,1,986.278778,...,0,-34.343833,223.112578,5,1,4,0.800000,0.200000,-33.024575,0
2,0003c76e33adfa3484f6f8426d6f840cdaaade5bf71a2a...,0,-0.070451,4057.439505,4813.635,3654.630,4,4,4,566.227093,...,0,-12.877833,126.482115,1,0,1,1.000000,0.000000,-17.594144,0
3,0003e52fe559c035a21c6b38b6f16612a6689ea555480e...,0,0.022626,4035.443736,1716.795,371.385,4,4,4,5367.771943,...,0,-14.949000,3896.685693,52,18,34,0.653846,0.346154,-82.188555,0
4,0004d0051f38ca5c9df27bca501d556fd16fa9acd2df39...,0,-0.051477,20042.205824,35462.130,6178.560,1,0,1,17251.215708,...,0,-325.373000,8107.574099,24,6,18,0.750000,0.250000,-387.819611,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50449,fff76a8dfb3e78fe545169ce8427012d783ab7f51e9fe1...,0,0.049060,13735.436538,9399.855,4564.485,1,1,4,9317.807627,...,0,-53.726333,4720.654310,35,8,27,0.771429,0.228571,-156.302523,0
50450,fff798878bc2bb60ead2b2c405bc1c4bdc1b1ff713a422...,0,0.018823,9551.249505,19041.975,6706.125,1,1,1,12446.873858,...,0,-137.065000,7312.832971,41,12,29,0.707317,0.292683,-249.323867,0
50451,fff7d14c2d27b7ba9c8c2e06940d38d17566b7ae04f208...,0,-0.252986,26218.259670,8337.045,60026.925,1,1,0,14137.128317,...,0,574.332000,8400.989160,44,7,37,0.840909,0.159091,295.022048,1
50452,fffae87c8a91a689bac61b5451f1f9887e681ccd723013...,0,0.150597,136050.193681,127841.040,129165.840,5,5,5,91483.292503,...,0,14.720000,29363.968737,78,21,57,0.730769,0.269231,-2420.956036,0


In [10]:
# # Export to csv if needed
# user_features_final_growth.to_csv("user_features_l90_growth_20231111.csv", index=False)
# user_features_final_stable.to_csv("user_features_l90_stable_20231111.csv", index=False)